## Generate feasible counterfactual explanations using a VAE

This presents the variational inference based approach for generating feasible counterfactuals, where we first train an encoder-decoder framework to generate counterfactuals. More details about our framework can be found here: https://arxiv.org/abs/1912.03277

In [ ]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers  # helper functions

%load_ext autoreload
%autoreload 2

DiCE requires two inputs: a training dataset and a pre-trained ML model. It can also work without access to the full dataset (see this [notebook](DiCE_with_private_data.ipynb) for advanced examples).

### Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in **dice_ml.utils.helpers** module. 

In [ ]:
dataset = helpers.load_adult_income_dataset()

This dataset has 8 features. The outcome is income which is binarized to 0 (low-income, <=50K) or 1 (high-income, >50K). 

In [ ]:
dataset.head()

In [ ]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

Given this dataset, we construct a data object for DiCE. Since continuous and discrete features have different ways of perturbation, we need to specify the names of the continuous features. DiCE also requires the name of the output variable that the ML model will predict.

In [ ]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'],
                 outcome_name='income', data_name='adult', test_size=0.1)

### Loading the ML model

Below, we use a pre-trained ML model which produces high accuracy comparable to other baselines. For convenience, we include the sample trained model with the DiCE package.

Note that we need to specify the explainer in the model backend. This is because both model and explainer need to be using the same backend library (pytorch or tensorflow).

In [ ]:
backend = {'model': 'pytorch_model.PyTorchModel',
           'explainer': 'feasible_base_vae.FeasibleBaseVAE'}
ML_modelpath = helpers.get_adult_income_modelpath(backend='PYT')
ML_modelpath = ML_modelpath[:-4] + '_2nodes.pth'
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)
m.load_model()
print('ML Model', m.model)

### Generate counterfactuals using a VAE model

Based on the data object *d* and the model object *m*, we can now instantiate the DiCE class for generating explanations. 
We present the variational inference based approach towards generating counterfactuals, where we first train an encoder-decoder framework to generate counterfactuals.

FeasibleBaseVAE class has an method `train()`, which would train the Variational Encoder Decoder framework on the input dataframe. It has another arugment, `pre_trained`, which if set to 0 would re-train the framework each time while generating CFs. Else, it can be set to 1 to avoid repeated training of the framework and would load the latest fitted VAE model. 

In [ ]:
# initiate DiCE
exp = dice_ml.Dice(d, m, encoded_size=10, lr=1e-2,
                   batch_size=2048, validity_reg=42.0, margin=0.165, epochs=25,
                   wm1=1e-2, wm2=1e-2, wm3=1e-2)
exp.train(pre_trained=1)

DiCE is a form of a local explanation and requires an query input whose outcome needs to be explained. Below we provide a sample input whose outcome is 0 (low-income) as per the ML model object *m*. 

In [ ]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age': 41,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

Given the query input, we can now generate counterfactual explanations to show perturbed inputs from the original input where the ML model outputs class 1 (high-income). 

In [ ]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")
# visualize the results
dice_exp.visualize_as_dataframe(show_only_changes=True)

That's it! You can try generating counterfactual explanations for other examples using the same code. You can compare the running time of this VAE-based to DiCE's default method: VAE-based method is super fast!

## Adding feasibility constraints
However, you might notice that for some examples, the above method can still return infeasible counterfactuals. This requires our base framework to be adpated for prodcuing feasible counterfactuals. A detailed description of how we adapt the method under different assumptions is provided in [this paper](https://arxiv.org/pdf/1912.03277). 

In the section below, we show an adaptation our base approach for preserving the Age-Ed constraint: Age and Education can never decrease and increasing Education implies increase in Age. This approach is called **ModelApprox**, where we adapt our base approach for simple unary and binary constraints. 

### ModelApprox 

Similar to the FeasibleBaseVAE class above, FeasibleModelApprox class has a method `train()` with argument `pre_trained`, which determines whether to train the framework again or load the latest optimal model. However, there are additional arguments to the `train()` method:

1. The first arugment determines whether the constraint to be preserved is unary or monotonic
2. The second arugment provides the list of constraint variable names: [[Effect, Cause_1,..,Cause_n]]. In the case of a unary constraint, there would be no causes but only a single constrained variable.
3. The third argument provides the intended direction of change for the constrained variables: Value of 1 means that we allow for only increase in the constrained variable on the change from data point to its counterfactual and vice versa.
4. The fourth argument refers to the penalty weight for infeasibility under given constraint. 

### Initilize the Model and Explainer for FeasibleModelApprox

In [ ]:
backend = {'model': 'pytorch_model.PyTorchModel',
           'explainer': 'feasible_model_approx.FeasibleModelApprox'}
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
ML_modelpath = ML_modelpath[:-4] + '_2nodes.pth'
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)
m.load_model()
print('ML Model', m.model)

In [ ]:
# initiate DiCE
exp = dice_ml.Dice(d, m, encoded_size=10, lr=1e-2, batch_size=2048,
                   validity_reg=76.0, margin=0.344, epochs=25,
                   wm1=1e-2, wm2=1e-2, wm3=1e-2)
exp.train(1, [[0]], 1, 87, pre_trained=1)

In [ ]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")
# visualize the results
dice_exp.visualize_as_dataframe(show_only_changes=True)

The results for ModelApprox show that the Age is also increased with increase in Education in counterfactual explanations unlike the BaseVAE method. You can try to experiment with ModelApprox to preserve unary and monotonic constraints for other datasets too. Examples for even more advanced approaches like **SCMGenCF**,**OracleGenCF** would be included soon to this repository, where we learn to generate feasible counterfactuals for complex feasiblity constraints. More details can be found in [our paper](https://arxiv.org/pdf/1912.03277).